In [ ]:
from datetime import datetime, timedelta
import json
from pathlib import Path
import pytz
import requests
import time
import pandas as pd

organizationId = 'XXXXXXXXXXXXXXXXXXXXXXX'
deliveryStatus1 = 'DELIVERED'
deliveryStatus2 = 'CLOSED'
request_timeout = '00:05:00'
df_second_year = pd.DataFrame()
df_first_year = pd.DataFrame()

user_id = 'XXXXXXXXXXXXX'
user_secret = 'XXXXXXXXXXXXXXXX'
url = f'https://iiko.biz:9900/api/0/auth/access_token?user_id={user_id}&user_secret={user_secret}'
access_token = requests.get(url).text.strip('"')

dateFrom = datetime(2022, 5, 16).strftime('%Y-%m-%d')
dateTo = datetime(2022, 5, 22).strftime('%Y-%m-%d')

#dateFrom = datetime(2022, 3, 1).strftime('%Y-%m-%d')
#dateTo = datetime(2022, 3, 31).strftime('%Y-%m-%d')

url = f'https://iiko.biz:9900/api/0/orders/deliveryOrders?access_token={access_token}&organization={organizationId}&dateFrom={dateFrom}&dateTo={dateTo}&deliveryStatus={deliveryStatus1}&deliveryStatus={deliveryStatus2}&request_timeout={request_timeout}'
json_data = json.loads(requests.get(url).text)

df_period = pd.json_normalize(json_data['deliveryOrders'])
df_period = df_period[['sum', 'comment']]

In [ ]:
import re
import math

def find_source(x):
    if x.find('DC') == -1:
        return 0
    else:
        return 1

def discount_delivery(x):
    if x == 0:
        return 0
    else:
        return x - 15

def find_discount(x):
    if re.findall("SALE(\d*)", x):
        return re.findall("SALE(\d*)", x)[0]
    else:
        return 0

def find_id(x):
    return re.findall("DC: ([\d]*-[\d]*-[\d]*)", x)[0]

df_period['DC'] = df_period['comment'].apply(find_source)
df_period = df_period[df_period['DC']==1]
df_period['discount'] = df_period['comment'].apply(find_discount)
df_period['id'] = df_period['comment'].apply(find_id)
df_period.sort_values(by='id', inplace=True)
df_period['sum'] = df_period['sum'].astype(float)
df_period['discount'] = df_period['discount'].astype(int) 
df_period['afterdiscount'] = df_period['sum'] * ((100 - df_period['discount']) / 100)
df_period['afterdiscount'] = df_period['afterdiscount'].apply(lambda x: math.ceil(x))
df_period['dicountdelivery'] = df_period['discount'].apply(discount_delivery)
df_period['discountamoung'] = df_period['sum'] - df_period['afterdiscount'] 
df_period['dicountdeliveryrubl'] = df_period['sum'] * (df_period['dicountdelivery'] / 100)

print('Приход после промокодов:', df_period['afterdiscount'].sum())
print("Комиссия 15%:",  round(df_period['afterdiscount'].sum()*0.15))
print('Сумма возврата промокодов:', df_period['dicountdeliveryrubl'].sum())
print('Итого мы получим:', round(df_period['afterdiscount'].sum()*0.85 + df_period['dicountdeliveryrubl'].sum()))